### RNN是Natural Language Processing(NLP)中常用的Neural Network
簡單的RNN模型(Vanilla RNN)額外考慮前文的關係，把簡單迴歸的模型 y=W*x+b
改成
#### h_t=W*h_(t-1)+Ux_t+b
#### y_t=g(V*h_t)
#### W，U，V都是權重，對每一個推導點都共用
#### 通常activation function是tanh
#### 隱藏層使用simpleRNN

##### 這裡的RNN都只考慮前一項字詞；有學者提出『雙向』(Bidirectional) RNN
##### Keras提供 Bidirectional() 函數
##### model.add(Bidirectional(SimpleRNN(10)))
##### https://www.slideshare.net/xavigiro/deep-learning-for-computer-vision-recurrent-neural-networks-upc-2016

##### RNN的缺點是因函數可展開如下:
##### h_t=W*h_(t-1)+Ux_t+b
##### =W*[W*h_(t-2)+Ux_(t-1)+b]+Ux_t+b
##### =[W^2*h_(t-2)+W*Ux_(t-1)+W*b]+Ux_t+b
#####  ...
##### h_t=W^t*h_0+...+Ux_t+b
##### 由上式可發現h_0對函數的影響是w的t次方，通常，w 會小於 1，因為 w 大於1，
##### 則表示距離越遠的字句影響力會遠大於距離近的字句，這不太合理，所以，w 應會小於 1，
##### 當 t 很大時，w 的 t 次方會趨近於0，也就是說，段落或句子很長，越前面的字詞會被遺忘，
##### 這種現象稱之為『梯度消失』(Vanishing Gradient)，

###### 後來Hochreiter & Schmidhuber) 在 1997 年提出『長短期記憶網路』(Long Short Term Memory Network, 
###### LSTM) ，透過記憶功能來增加『長期依賴』(long-term dependency)的問題。
##### 實作上將RNN隱藏層由SimpleRNN layer 改成 LSTM即可
##### LSTM應用很廣，其中{情緒分析}(Sentiment Analysis)，看看如何分析影評的正/負意見分類，可用在網路，產品調查等。

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import torch.nn.functional as F

### 使用 MNIST 資料集 (載入資料)

In [1]:
train_dataset = dsets.MNIST(
    root = './t10k-labels.idx1-ubyte',
    train = True,
    transform = transforms.ToTensor(),
    download = True
)
test_dataset = dsets.MNIST(
    root = './MNIST_data',
    train = False,
    transform = transforms.ToTensor()
)

NameError: name 'dsets' is not defined

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RNNModel, self).__init__()   
        
        self.rnn = nn.RNN(
            input_size = input_dim,
            hidden_size = 100,
            num_layers = 1,
            batch_first=True,
            nonlinearity='relu')
        
        self.fc = nn.Linear(100, 10)
        
    def forward(self, x):
        # (layer_dim, batch_size, hidden_dim)
        h0 = None
        # another hidden state example :
        # h0 = Variable(torch.zeros(1, x.size(0), 100))
        
        out, hn = self.rnn(x, h0)
        # "out" dim : (100, 28, 100)
        # "-1" means the last time step
        
        out = self.fc(out[:, -1, :])  # (100, 100)  #這裡的-1可以用別的
        # "out" dim : (100, 10)
        
        return out